In [1]:
import pandas as pd
import numpy as np

from pybaseball import statcast

In [2]:
dates = {2015: ['2015-04-05', '2015-10-04'],
        2016: ['2016-04-03', '2016-10-02'],
        2017: ['2017-04-02', '2017-10-01'],
        2018: ['2018-03-29', '2018-10-01'],
        2019: ['2019-03-28', '2019-09-29']}

In [4]:
all_data = pd.DataFrame()
for v in dates.values():
    tmp = statcast(start_dt=v[0], end_dt=v[1])
    all_data = pd.concat([all_data, tmp])

This is a large query, it may take a moment to complete
Completed sub-query from 2015-04-05 to 2015-04-10
Completed sub-query from 2015-04-11 to 2015-04-16
Completed sub-query from 2015-04-17 to 2015-04-22
Completed sub-query from 2015-04-23 to 2015-04-28
Completed sub-query from 2015-04-29 to 2015-05-04
Completed sub-query from 2015-05-05 to 2015-05-10
Completed sub-query from 2015-05-11 to 2015-05-16
Completed sub-query from 2015-05-17 to 2015-05-22
Completed sub-query from 2015-05-23 to 2015-05-28
Completed sub-query from 2015-05-29 to 2015-06-03
Completed sub-query from 2015-06-04 to 2015-06-09
Completed sub-query from 2015-06-10 to 2015-06-15
Completed sub-query from 2015-06-16 to 2015-06-21
Completed sub-query from 2015-06-22 to 2015-06-27
Completed sub-query from 2015-06-28 to 2015-07-03
Completed sub-query from 2015-07-04 to 2015-07-09
Completed sub-query from 2015-07-10 to 2015-07-15
Completed sub-query from 2015-07-16 to 2015-07-21
Completed sub-query from 2015-07-22 to 2015-

In [5]:
deprecated = [col for col in all_data.columns if 'deprecated' in col]
all_data = all_data.drop(columns=deprecated)

In [6]:
all_data = all_data.drop(columns=['spin_dir', 'umpire', 'index'])
all_data = all_data.rename(columns={'pitcher.1': 'pitcher_1', 'fielder_2.1': 'fielder_2_1'})
all_data = all_data.replace({np.nan: None})

In [7]:
season_dict = {
    2015: all_data[all_data['game_year']==2015],
    2016: all_data[all_data['game_year']==2016],
    2017: all_data[all_data['game_year']==2017],
    2018: all_data[all_data['game_year']==2018],
    2019: all_data[all_data['game_year']==2019]
}

In [8]:
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [29]:
conn = connect(
    user = "",
    host = "localhost",
    password = ""
)

In [30]:
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()

In [31]:
cur.execute("CREATE DATABASE baseball")

In [32]:
cur.close()
conn.close()

In [75]:
int_fields = ['batter', 'pitcher', 'zone', 'hit_location', 'balls', 'strikes', 'game_year', 'on_1b', 'on_2b', 'on_3b',
             'outs_when_up', 'inning', 'fielder_2', 'game_pk', 'pitcher_1', 'fielder_2_1', 'fielder_3', 'fielder_4',
             'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'launch_speed_angle', 'at_bat_number',
              'pitch_number','home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score', 'post_home_score',
              'post_bat_score', 'post_fld_score']
float_fields = ['release_speed', 'release_pos_x', 'release_pos_z', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'hc_x', 'hc_y',
               'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_angle', 'launch_speed',
               'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y', 'estimated_ba_using_speedangle',
               'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value']
# all_data[int_fields] = all_data[int_fields].astype('Int64')

In [76]:
create_table_str = 'CREATE TABLE IF NOT EXISTS statcast('
for i, d in enumerate(all_data.columns):
    if d in int_fields:
        create_table_str += f'{d} INT'
    elif d in float_fields:
        create_table_str += f'{d} FLOAT'
    elif all_data[d].dtype == 'datetime64[ns]':
        create_table_str += f'{d} DATE'
    else:
        create_table_str += f'{d} TEXT'        
    
    if i < len(all_data.columns) - 1:
        create_table_str += ','
create_table_str += ');'

In [78]:
conn = connect("dbname=baseball user= password=")
cur = conn.cursor()

In [79]:
cur.execute(create_table_str)

In [80]:
cols = ", ".join([str(i) for i in season_dict[2015].columns.tolist()])
cols

'pitch_type, game_date, release_speed, release_pos_x, release_pos_z, player_name, batter, pitcher, events, description, zone, des, game_type, stand, p_throws, home_team, away_team, type, hit_location, bb_type, balls, strikes, game_year, pfx_x, pfx_z, plate_x, plate_z, on_3b, on_2b, on_1b, outs_when_up, inning, inning_topbot, hc_x, hc_y, fielder_2, sv_id, vx0, vy0, vz0, ax, ay, az, sz_top, sz_bot, hit_distance_sc, launch_speed, launch_angle, effective_speed, release_spin_rate, release_extension, game_pk, pitcher_1, fielder_2_1, fielder_3, fielder_4, fielder_5, fielder_6, fielder_7, fielder_8, fielder_9, release_pos_y, estimated_ba_using_speedangle, estimated_woba_using_speedangle, woba_value, woba_denom, babip_value, iso_value, launch_speed_angle, at_bat_number, pitch_number, pitch_name, home_score, away_score, bat_score, fld_score, post_away_score, post_home_score, post_bat_score, post_fld_score, if_fielding_alignment, of_fielding_alignment'

In [81]:
# from psycopg2.extensions import register_adapter, AsIs
# def adapt_numpy_int64(numpy_int64):
#     return AsIs(numpy_int64)
# register_adapter(np.int64, adapt_numpy_int64)

In [82]:
for year in range(2015, 2020):
    for i, row in season_dict[year].iterrows():
        query = "INSERT INTO statcast (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cur.execute(query, tuple(row))
        conn.commit()
        if ((i+1) % 10000 == 0):
            print(f'Number of rows added: {i+1} for {year}.')

Number of rows added: 10000 for 2015.
Number of rows added: 20000 for 2015.
Number of rows added: 30000 for 2015.
Number of rows added: 40000 for 2015.
Number of rows added: 50000 for 2015.
Number of rows added: 60000 for 2015.
Number of rows added: 70000 for 2015.
Number of rows added: 80000 for 2015.
Number of rows added: 90000 for 2015.
Number of rows added: 100000 for 2015.
Number of rows added: 110000 for 2015.
Number of rows added: 120000 for 2015.
Number of rows added: 130000 for 2015.
Number of rows added: 140000 for 2015.
Number of rows added: 150000 for 2015.
Number of rows added: 160000 for 2015.
Number of rows added: 170000 for 2015.
Number of rows added: 180000 for 2015.
Number of rows added: 190000 for 2015.
Number of rows added: 200000 for 2015.
Number of rows added: 210000 for 2015.
Number of rows added: 220000 for 2015.
Number of rows added: 230000 for 2015.
Number of rows added: 240000 for 2015.
Number of rows added: 250000 for 2015.
Number of rows added: 260000 for 2

Number of rows added: 710000 for 2017.
Number of rows added: 720000 for 2017.
Number of rows added: 10000 for 2018.
Number of rows added: 20000 for 2018.
Number of rows added: 30000 for 2018.
Number of rows added: 40000 for 2018.
Number of rows added: 50000 for 2018.
Number of rows added: 60000 for 2018.
Number of rows added: 70000 for 2018.
Number of rows added: 80000 for 2018.
Number of rows added: 90000 for 2018.
Number of rows added: 100000 for 2018.
Number of rows added: 110000 for 2018.
Number of rows added: 120000 for 2018.
Number of rows added: 130000 for 2018.
Number of rows added: 140000 for 2018.
Number of rows added: 150000 for 2018.
Number of rows added: 160000 for 2018.
Number of rows added: 170000 for 2018.
Number of rows added: 180000 for 2018.
Number of rows added: 190000 for 2018.
Number of rows added: 200000 for 2018.
Number of rows added: 210000 for 2018.
Number of rows added: 220000 for 2018.
Number of rows added: 230000 for 2018.
Number of rows added: 240000 for 2

In [83]:
cur.close()
conn.close()